# Exercise 1: Elastic rotating bar

In [1]:
from ngsolve import *
from netgen.occ import *
import ipywidgets as widgets
from ngsolve.comp import GlobalSpace
from ngsolve.webgui import Draw
from ngsolve.solvers import Newton
from ngsolve.comp import DifferentialSymbol
import numpy as np
# ngsglobals.msg_level=10
import time

In [2]:
shape = Rectangle(1, 0.1).Face()

# move shape's center of mass to origin; set start translation accordingly
r = (shape.center[0], shape.center[1])
shape = shape.Move((-r[0], -r[1], 0))

mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=0.05))
Draw(mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [3]:
moment_of_inertia = Integrate((x**2 + y**2)*dx, mesh)
mass = Integrate(CF(1)*dx, mesh)

## time-stepping

Variables are

* position $\phi = r + R x$, an affine linear function, with constraint $R^T R = I$, i.e. $R$ is orthogonal (and thus, by continuity, a rotation matrix)
* velocity $v = a + b \wedge x$, in body-frame

In [4]:
E, nu = 210, 0.2
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

def Stress(strain):
    return 2*mu*strain + lam*Trace(strain)*Id(2)  

In [5]:
tau = 0.005
tend = 2
u0 = exp(-400*(x**2 + y**2)) 
v0 = 0

fes = VectorH1(mesh, order=3)
u,testf = fes.TnT()
aform = InnerProduct(Stress(Sym(Grad(u))), Sym(Grad(testf)))*dx

a = BilinearForm(aform).Assemble()
mstar = BilinearForm(tau**2/4*aform).Assemble()
mstarinv = mstar.mat.Inverse(inverse="sparsecholesky")
f = LinearForm(fes).Assemble()

In [6]:
gfu = GridFunction(fes)
gfv = GridFunction(fes)

# gfu.Set(u0)
gfv.Set(CF(((y - shape.center[1]), -(x - shape.center[0]))))

scene = Draw (gfu, deformation=True, order=3)

for j in range(int(tend/tau)):
    gfu.vec.data += tau/2 * gfv.vec
    gfv.vec.data += tau * mstarinv * (f.vec - a.mat * gfu.vec)
    gfu.vec.data += tau/2 * gfv.vec

    scene.Redraw()
    time.sleep(0.5)
scene.Redraw()


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

KeyboardInterrupt: 